In [2]:
import duckdb
import numpy as np

In [46]:
con = duckdb.connect(database='jus.duckdb', read_only=False)

In [44]:
con.execute(f"CREATE TABLE BigBird(id INT PRIMARY KEY, {', '.join([f'col_{j} FLOAT8' for j in range(4)])});")

In [49]:
con.execute("CREATE TABLE BigBird(id INT PRIMARY KEY, data FLOAT8[]);")

In [72]:
con.execute("CREATE TABLE BigBird_norm(id INT UNIQUE REFERENCES BigBird(id), norm FLOAT8);")

CatalogException: Catalog Error: Table with name "BigBird_norm" already exists!

In [ ]:
concedendo_label = 1
negando_label = 0

In [60]:
data = con.execute("SELECT * FROM BigBird").fetchnumpy()
data

{'id': array([0, 1, 2, 3], dtype=int32),
 'data': array([list([25.0, 3.0, 3.0, 1.0]), list([29.0, 5.0, 4.0, 0.0]),
        list([30.0, 6.0, -3.0, 0.0]), list([1.0, 0.0, 0.0, 1.0])],
       dtype=object)}

In [77]:
import pandas as pd
import numpy as np

# Exemplo de conjunto de dados
dados = {
    'x': [25, 29, 30,1],
    'y': [3, 5, 6,0],
    'z': [3, 4, -3,0],
    'k': [1,0,0,1]
}
dados_selecionados = pd.DataFrame(dados)

def cosine_similarity(A, B):

    # Calcula o produto escalar
    dot_product = np.dot(A, B)
    
    # Calcula as normas dos vetores
    norm_A = np.linalg.norm(A)
    norm_B = np.linalg.norm(B)
    
    if norm_A == 0 or norm_B == 0:
        return 0.0
    
    # Calcula a similaridade de cosseno
    cosine_sim = dot_product / (norm_A * norm_B)
    
    return cosine_sim

# Calculando a similaridade entre candidatos
num_candidatos = len(dados_selecionados)
similaridades = np.zeros((num_candidatos, num_candidatos))

for i in range(num_candidatos):
    for j in range(i+1, num_candidatos):  # começar a partir de i+1 
        sim = cosine_similarity(dados_selecionados.iloc[i], dados_selecionados.iloc[j])
        similaridades[i][j] = sim
        similaridades[j][i] = sim  # preenchendo a matriz simétrica

similaridades

array([[0.        , 0.99779337, 0.97293301, 0.72446168],
       [0.99779337, 0.        , 0.97266384, 0.69047619],
       [0.97293301, 0.97266384, 0.        , 0.69006556],
       [0.72446168, 0.69047619, 0.69006556, 0.        ]])

In [37]:
con.execute("""
INSERT INTO BigBird (id,col_0,col_1,col_2,col_3) VALUES
	(0,25.0,3.0,3.0,1.0),
	(1,29.0,5.0,4.0,0.0),
	(2,30.0,6.0,-3.0,0.0),
  (3,1.0,0.0,0.0,1.0)
;
""")

In [38]:
con.execute(f"""
    SELECT 
        id,
        SQRT(
            {'+'.join([f'col_{j}*col_{j}' for j in range(4)])}
        ) AS norm
    FROM BigBird         
""").fetchnumpy()

{'id': array([0, 1, 2, 3], dtype=int32),
 'norm': array([25.37715508, 29.69848481, 30.7408523 ,  1.41421356])}

In [24]:
con.execute(f"""
WITH norms AS (
    SELECT 
        id,
        SQRT(
            {'+'.join([f'col_{j}*col_{j}' for j in range(4)])}
        ) AS norm
    FROM BigBird
)

INSERT INTO BigBird_norm (id, norm)
SELECT id, norm
FROM norms

ON CONFLICT (id) 
DO UPDATE SET norm = EXCLUDED.norm;            
""").fetchnumpy()

{'Count': masked_array(data=[4],
              mask=[False],
        fill_value=999999)}

In [50]:
con.execute("""
INSERT INTO BigBird (id,data) VALUES
	(0,ARRAY[25.0,3.0,3.0,1.0]),
	(1,ARRAY[29.0,5.0,4.0,0.0]),
	(2,ARRAY[30.0,6.0,-3.0,0.0]),
  (3,ARRAY[1.0,0.0,0.0,1.0])
;
""")

In [70]:
con.execute("""
SELECT 
    id,
    SQRT(SUM(POWER(val, 2))) AS norm
FROM 
    BigBird,
    LATERAL UNNEST(data) AS t(val)
GROUP BY 
    id;
""").fetchnumpy()

{'id': array([0, 1, 2, 3], dtype=int32),
 'norm': array([25.37715508, 29.69848481, 30.7408523 ,  1.41421356])}

In [73]:
con.execute("""
WITH norms AS (
  SELECT 
      id,
      SQRT(SUM(POWER(val, 2))) AS norm
  FROM 
      BigBird,
      LATERAL UNNEST(data) AS t(val)
  GROUP BY 
      id
)
INSERT INTO BigBird_norm (id, norm)
SELECT id, norm
FROM norms

ON CONFLICT (id) 
DO UPDATE SET norm = EXCLUDED.norm; 
""").fetchnumpy()  

{'Count': masked_array(data=[4],
              mask=[False],
        fill_value=999999)}

In [74]:
con.execute("SELECT * FROM BigBird_norm").fetchnumpy()

{'id': array([0, 1, 2, 3], dtype=int32),
 'norm': array([25.37715508, 29.69848481, 30.7408523 ,  1.41421356])}

In [41]:
print(f"""SELECT 
    a.id AS idA,
    b.id AS idB,
    SUM(
        {'+'.join([f'a.col_{j}*b.col_{j}' for j in range(4)])}
        ) AS dot_product
FROM 
    BigBird a
CROSS JOIN 
    BigBird b
WHERE 
    a.id < b.id
GROUP BY 
    a.id, b.id""")

SELECT 
    a.id AS idA,
    b.id AS idB,
    SUM(
        a.col_0*b.col_0+a.col_1*b.col_1+a.col_2*b.col_2+a.col_3*b.col_3
        ) AS dot_product
FROM 
    BigBird a
CROSS JOIN 
    BigBird b
WHERE 
    a.id < b.id
GROUP BY 
    a.id, b.id


In [75]:
con.execute("""
WITH expanded AS (
    SELECT
        a.id AS idA,
        b.id AS idB,
        unnest(a.data) AS a_data,
        unnest(b.data) AS b_data,
        row_number() OVER(PARTITION BY a.id, b.id ORDER BY a.id) as rn
    FROM
        BigBird a
    CROSS JOIN
        BigBird b
    WHERE
        a.id < b.id
)

SELECT
    idA,
    idB,
    sum(a_data * b_data) AS dot_product
FROM
    expanded
GROUP BY
    idA, idB
""").fetchnumpy()

{'idA': array([0, 0, 0, 1, 1, 2], dtype=int32),
 'idB': array([1, 2, 3, 2, 3, 3], dtype=int32),
 'dot_product': array([752., 759.,  26., 888.,  29.,  30.])}

In [76]:
con.execute("""
WITH DotProducts AS (
    WITH expanded AS (
        SELECT
            a.id AS idA,
            b.id AS idB,
            unnest(a.data) AS a_data,
            unnest(b.data) AS b_data,
            row_number() OVER(PARTITION BY a.id, b.id ORDER BY a.id) as rn
        FROM
            BigBird a
        CROSS JOIN
            BigBird b
        WHERE
            a.id < b.id
    )

    SELECT
        idA,
        idB,
        sum(a_data * b_data) AS dot_product
    FROM
        expanded
    GROUP BY
        idA, idB
                
    )
SELECT 
    d.idA,
    d.idB,
    CASE 
        WHEN n1.norm = 0 OR n2.norm = 0 THEN 0
        ELSE d.dot_product / (n1.norm * n2.norm)
    END AS cosine_similarity
FROM 
    DotProducts d
JOIN 
    BigBird_norm n1 ON d.idA = n1.id
JOIN 
    BigBird_norm n2 ON d.idB = n2.id;
""").fetchnumpy()

{'idA': array([0, 0, 0, 1, 1, 2], dtype=int32),
 'idB': array([1, 2, 3, 2, 3, 3], dtype=int32),
 'cosine_similarity': array([0.99779337, 0.97293301, 0.72446168, 0.97266384, 0.69047619,
        0.69006556])}

In [78]:
similaridades

array([[0.        , 0.99779337, 0.97293301, 0.72446168],
       [0.99779337, 0.        , 0.97266384, 0.69047619],
       [0.97293301, 0.97266384, 0.        , 0.69006556],
       [0.72446168, 0.69047619, 0.69006556, 0.        ]])

In [31]:
query_cosine_similarity = f"""
WITH DotProducts AS (
    SELECT 
        a.id AS idA,
        b.id AS idB,
        SUM(
            {'+'.join([f'a.col_{j}*b.col_{j}' for j in range(4)])}
            ) AS dot_product
    FROM 
        BigBird a
    CROSS JOIN 
        BigBird b
    WHERE 
        a.id < b.id
    GROUP BY 
        a.id, b.id
)
SELECT 
    d.idA,
    d.idB,
    CASE 
        WHEN n1.norm = 0 OR n2.norm = 0 THEN 0
        ELSE d.dot_product / (n1.norm * n2.norm)
    END AS cosine_similarity
FROM 
    DotProducts d
JOIN 
    BigBird_norm n1 ON d.idA = n1.id
JOIN 
    BigBird_norm n2 ON d.idB = n2.id;
"""
print(query_cosine_similarity)


WITH DotProducts AS (
    SELECT 
        a.id AS idA,
        b.id AS idB,
        SUM(
            a.col_0*b.col_0+a.col_1*b.col_1+a.col_2*b.col_2+a.col_3*b.col_3
            ) AS dot_product
    FROM 
        BigBird a
    CROSS JOIN 
        BigBird b
    WHERE 
        a.id < b.id
    GROUP BY 
        a.id, b.id
)
SELECT 
    d.idA,
    d.idB,
    CASE 
        WHEN n1.norm = 0 OR n2.norm = 0 THEN 0
        ELSE d.dot_product / (n1.norm * n2.norm)
    END AS cosine_similarity
FROM 
    DotProducts d
JOIN 
    BigBird_norm n1 ON d.idA = n1.id
JOIN 
    BigBird_norm n2 ON d.idB = n2.id;



In [32]:
con.execute(query_cosine_similarity).fetchnumpy()

{'idA': array([0, 0, 0, 1, 1, 2], dtype=int32),
 'idB': array([1, 2, 3, 2, 3, 3], dtype=int32),
 'cosine_similarity': array([0.99779337, 0.97293301, 0.72446168, 0.97266384, 0.69047619,
        0.69006556])}

In [35]:
similaridades

array([[0.        , 0.99779337, 0.97293301, 0.72446168],
       [0.99779337, 0.        , 0.97266384, 0.69047619],
       [0.97293301, 0.97266384, 0.        , 0.69006556],
       [0.72446168, 0.69047619, 0.69006556, 0.        ]])

In [79]:
con.execute("drop table BigBird_norm")

In [80]:
con.execute("drop table BigBird")

In [45]:
con.close()